In [2]:
import pandas as pd
from deep_translator import GoogleTranslator
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier
import pickle

In [630]:
data = pd.read_csv('train.csv')  # загурзка тестового датасета

In [16]:
gtranslate_china = GoogleTranslator(source='auto', target='russian')  # объявление переводчика
tokenizer = RegexpTokenizer(r'\w+')  # объявление токенизатора
morph = pymorphy2.MorphAnalyzer()  # объявление лемматизатора
stop_words = stopwords.words('russian')  # стоп-слова

In [632]:
# Преобразование тренировочного датасета №1
k = 0
for i in data["utterance"]:
    data["utterance"][k] = tokenizer.tokenize(gtranslate_china.translate(i))  # перевод и токенизация
    data["utterance"][k] = [morph.parse(word)[0][2] for word in data["utterance"][k]]  # лемматизация
    print(k, data["utterance"][k])
    k += 1

0 ['можно', 'ли', 'отменить', 'сделать', 'я', 'заказ']
1 ['отмена', 'заказ']
2 ['я', 'нужный', 'помощь', 'в', 'отмена', 'последний', 'заказ', 'который', 'я', 'сделать']
3 ['проблема', 'с', 'отмена', 'заказ', 'который', 'я', 'сделать']
4 ['я', 'не', 'знать', 'как', 'отменить', 'заказ', 'который', 'я', 'сделать']
5 ['вы', 'мочь', 'помочь', 'я', 'отменить', 'заказ', 'который', 'я', 'сделать']
6 ['я', 'хотеть', 'бы', 'знать', 'о', 'отмена', 'заказ']
7 ['не', 'мочь', 'бы', 'вы', 'помочь', 'я', 'отменить', 'заказ']
8 ['я', 'не', 'знать', 'как', 'отменить', 'заказ', 'который', 'я', 'сделать']
9 ['помочь', 'я', 'отменить', 'мой', 'последний', 'заказ']
10 ['я', 'не', 'знать', 'как', 'отменить', 'последний', 'заказ', 'который', 'я', 'сделать']
11 ['я', 'нужный', 'помощь', 'с', 'отмена', 'заказ', 'который', 'я', 'сделать']
12 ['информация', 'о', 'отмена', 'заказ']
13 ['я', 'хотеть', 'бы', 'отменить', 'заказ']
14 ['я', 'нужный', 'помощь', 'в', 'отмена', 'мой', 'заказ']
15 ['я', 'нужный', 'помощь',

In [640]:
# Преобразование тренировочного датасета №2
for i in data["utterance"]:
    for j in i:
        if j in stop_words:
            i.remove(j)  # удаление стоп-слов
            print(k, j)
            k += 1

In [641]:
# Преобразование тренировочного датасета №3
k = 0
for i in data["utterance"]:
    data["utterance"][k] = " ".join(
        str(x) for x in data["utterance"][k])  # преобразование списка строк в строку
    k += 1

In [98]:
k = 0
while k != 6538:
    if 'получить возврат' in data["utterance"][k]:
        data["utterance"][k] = data["utterance"][k].replace('получить возврат','оформить возврат') # исправление косяков
        k += 1
    else:
        k += 1

In [99]:
# словарь с группами тем
dicc = {'create_account': 'чтобы создать аккаунт создайте аккаунт',
        'delete_account': 'чтобы удалить аккаунт, удалите аккаунт',
        'edit_account': 'чтобы редактировать аккаут, редактируйте аккаунт',
        'recover_password': 'чтобы восстановить пароль, восставте пароль',
        'registration_problems': 'проблемы с регистрацией? вот ответ',
        'switch_account': 'чтобы сменить аккаунт, смените аккаунт',
        'check_cancellation_fee': 'чтобы проверить оплату, проверьте оплату',
        'contact_customer_service': 'чтобы связаться с техподдержкой, обратитесь в техподдержку',
        'contact_human_agent': 'чтобы связаться с оператором, свяжитесь с оператором',
        'delivery_options': 'чтобы узнать опции доставки, узнайте',
        'delivery_period': 'срок доставки — 7-8 дней', 'complaint': 'ну давай пожалуйся ябида',
        'review': 'отзыв? оставьте ниже',
        'check_invoice': 'чтобы проверить счет, проверьте счет', 'get_invoice': 'получите чек',
        'newsletter_subscription': 'чтобы подписаться на новости, подпишитесь на новости',
        'cancel_order': 'чтобы отменить заказ, отмените заказ', 'change_order': 'чтобы изменить заказ, изменить заказ',
        'place_order': 'чтобы разместить заказ, разместите заказ',
        'track_order': 'чтобы отследить заказ, отследите заказ',
        'check_payment_methods': 'чтобы узнать способы оплаты, узнайте', 'payment_issue': 'проблема с оплатой? оплати',
        'check_refund_policy': 'политика возврата отсутствует', 'get_refund': 'возврата не будет',
        'track_refund': 'отследить ваш возврат нельзя',
        'change_shipping_address': 'нельзя сменить адрес доставки',
        'set_up_shipping_address': 'ваш адрес доставки установлен по умолчанию — Коммунистический 11'}

In [100]:
# Преобразование тренировочного датасета №4
data["answer"] = data["intent"].map(dicc)  # добавление нового столбца

In [83]:
data.to_csv(r'train_data.csv')  # экспорт преобразованного датасета

In [70]:
data = pd.read_csv('train_data.csv')  # импорт датасета

In [84]:
tfidfconverter = TfidfVectorizer()  # объявление векторайзера

In [102]:
X_train = tfidfconverter.fit_transform(data["utterance"]).toarray()  # векторизация x_train
y_train = data["answer"]

In [105]:
classifier = PassiveAggressiveClassifier()  # классификатор и его настройки

In [106]:
classifier.fit(X_train, y_train)  # обучение модели

PassiveAggressiveClassifier()

In [89]:
test_data = pd.read_csv('test.csv')  # загружаем тестовый датасет

In [17]:
# Преобразование тестового датасета №1
k = 0
for i in test_data["utterance"]:
    test_data["utterance"][k] = tokenizer.tokenize(gtranslate_china.translate(i))  # перевод и токенизация
    test_data["utterance"][k] = [morph.parse(word)[0][2] for word in test_data["utterance"][k]]  # лемматизация
    k += 1

In [22]:
# Преобразование тестового датасета №2
for i in test_data["utterance"]:
    for j in i:
        if j in stop_words:
            i.remove(j)  # удаление стоп-слов
            print(k, j)
            k += 1

In [23]:
# Преобразование тестового датасета №3

k = 0
for i in test_data["utterance"]:
    test_data["utterance"][k] = " ".join(
        str(x) for x in test_data["utterance"][k])  # преобразование списка строк в строку
    k += 1

In [24]:
# Преобразование тестового датасета №4
test_data["answer"] = test_data["intent"].map(dicc)  # создание нового столбца

In [628]:
test_data.to_csv(r'test_data.csv')

In [107]:
test_data = pd.read_csv('test_data.csv')

In [109]:
X_test = test_data["utterance"]
X_test = tfidfconverter.transform(X_test).toarray()
y_test = test_data["answer"]

In [111]:
y_pred = classifier.predict(X_test)  # попытка предсказывания

In [112]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))  # вывод проверки точности

                                                                  precision    recall  f1-score   support

ваш адрес доставки установлен по умолчанию — Коммунистический 11       1.00      1.00      1.00        32
                                               возврата не будет       1.00      0.93      0.96        28
                                   нельзя сменить адрес доставки       1.00      1.00      1.00        41
                                        ну давай пожалуйся ябида       1.00      1.00      1.00        23
                                            отзыв? оставьте ниже       1.00      1.00      1.00        32
                                    отследить ваш возврат нельзя       0.93      1.00      0.97        28
                                   политика возврата отсутствует       1.00      1.00      1.00        37
                                                    получите чек       1.00      1.00      1.00        31
                                      проблем

In [159]:
with open('model5.pkl','wb') as f: # выгрузка модели
        pickle.dump(classifier,f)